# This Notebook Concludes our Solution Developments on Forecasting

In [1]:
import numpy as np
from numpy import distutils
import distutils
import pandas as pd
from methods import read_pickle
from methods import find_index_v2
from methods import RM_model
from methods import NN_model
from methods import check_loads_plot
from sklearn.svm import SVR
from numpy import distutils
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.ensemble import VotingRegressor
import plotly.graph_objects as go
import keras
data_cleanned_v2 = read_pickle(path_name = 'data\data_processed_phase2.pickle')
data_cleanned_v3 = read_pickle(path_name = 'data\data_processed_experiment.pickle')

## The related buildings energy demand forecasting we made
### The MASE error we calculated was a simplified version which only took 

In [2]:
def split_training_dataframe(dict_df, training_type='Solar0', training_length=24*4*31, prediction_length=24*4*31):
    
    df = dict_df[training_type].copy()
    df['YEAR'] = df.index.year
    df['MONTH'] = df.index.month
    df['DATE'] = df.index.day
    df['HOUR'] = df.index.hour
    df['TIME'] = df.index.time
    df['WEEKDAY'] = df.index.weekday
    df['ISWEEKEND'] = np.where((df['WEEKDAY'] == 5) | (df['WEEKDAY'] == 6), True, False)
    if training_type.find('Building') != -1:
        df['LAG_DATA'] = df['consumption'].shift(24*4*31)
    df_training = df[-training_length - prediction_length: -prediction_length]
    
    # This if is to predict unseen data. Ask Nam personally if you are interested
    if prediction_length == 0:
        prediction_length = training_length
        
    df_prediction = df[-prediction_length:]
    
    return df_training, df_prediction, df

def get_training_parameter(df_training, training_type):    
    # Training data
    dew_tempC_tr = df_training['dewpoint_temperature (degC)']
    tempC_tr = df_training['temperature (degC)']
    radiation_tr = df_training['surface_solar_radiation (W/m^2)']
    thermal_radiation_tr = df_training['surface_thermal_radiation (W/m^2)']
    cloudcover_tr = df_training['total_cloud_cover (0-1)']
    humidity_tr = df_training['relative_humidity ((0-1))']
    hour_tr = df_training['HOUR'].astype('category').cat.codes.astype(float)
    weekday_tr = df_training['WEEKDAY'].astype('category').cat.codes.astype(float)
    time_tr = df_training['TIME'].astype('category').cat.codes.astype(float)   
    if training_type.find('Building') != -1:
        y_training = np.array(df_training['consumption'])
        lag_data_tr = df_training['LAG_DATA']
        occup_tr = df_training['occupancy (0-1)']
        x_training_RF = np.column_stack((occup_tr, tempC_tr, humidity_tr, cloudcover_tr, radiation_tr, lag_data_tr, time_tr, weekday_tr))    
    if training_type.find('Solar') != -1:
        y_training = np.array(df_training[training_type])
        x_training_RF = np.column_stack((cloudcover_tr, radiation_tr, time_tr))    
    return x_training_RF, y_training

def get_prediction_parameter(df_prediction, training_type):    
    # Predicting data  
    dew_tempC_pr = df_prediction['dewpoint_temperature (degC)']
    tempC_pr = df_prediction['temperature (degC)']
    radiation_pr = df_prediction['surface_solar_radiation (W/m^2)']
    # thermal_radiation_pr = df_prediction['surface_thermal_radiation (W/m^2)']
    cloudcover_pr = df_prediction['total_cloud_cover (0-1)']
    humidity_pr = df_prediction['relative_humidity ((0-1))']
    # hour_pr = df_prediction['HOUR'].astype('category').cat.codes.astype(float)
    weekday_pr = df_prediction['WEEKDAY'].astype('category').cat.codes.astype(float)
    time_pr = df_prediction['TIME'].astype('category').cat.codes.astype(float)
    
    if training_type.find('Building') != -1:
        y_real = df_prediction['consumption'].values
        lag_data_pr = df_prediction['LAG_DATA']
        occup_pr = df_prediction['occupancy (0-1)']
        x_prediction_RF = np.column_stack((occup_pr, tempC_pr, humidity_pr, cloudcover_pr, radiation_pr, lag_data_pr, time_pr, weekday_pr))
    
    if training_type.find('Solar') != -1:
        y_real = df_prediction[training_type].values
        x_prediction_RF = np.column_stack((cloudcover_pr, radiation_pr, time_pr))   
    return x_prediction_RF, y_real

def get_Nov_parameter(df, training_type, prediction_length, occu = False):    
    # Predicting data  
    length = prediction_length
    weather = df["weather"][:length]
    dew_tempC_oct = weather['dewpoint_temperature (degC)'][:length]
    tempC_oct = weather['temperature (degC)'][:length]
    radiation_oct = weather['surface_solar_radiation (W/m^2)'][:length]
    # thermal_radiation_pr = df_prediction['surface_thermal_radiation (W/m^2)']
    cloudcover_oct = weather['total_cloud_cover (0-1)']
    humidity_oct = weather['relative_humidity ((0-1))']
    # hour_pr = df_prediction['HOUR'].astype('category').cat.codes.astype(float)
    weekday_oct = pd.CategoricalIndex(weather.index.weekday).codes.astype(float)
    time_oct = pd.CategoricalIndex(weather.index.time).codes.astype(float)
    ## Summer time switch
    time_oct[252:] = (time_oct[252:]+4)%96
    
    if training_type.find('Building') != -1:
        lag_data_oct = df[training_type]['consumption'].values[-length:]
        occup_oct = df['occupancy'][:length]
        x_prediction_RF = np.column_stack((occup_oct, tempC_oct, humidity_oct, cloudcover_oct, radiation_oct, lag_data_oct, time_oct, weekday_oct)) if occu else np.column_stack((tempC_oct, humidity_oct, cloudcover_oct, radiation_oct, lag_data_oct, time_oct, weekday_oct))
    
    if training_type.find('Solar') != -1:
        lag_data_oct = df[training_type][training_type].values[-length:]
        x_prediction_RF = np.column_stack((cloudcover_oct, radiation_oct, time_oct))   
    return x_prediction_RF

def RF_training(x_training_RF, y_training):
    rf = RandomForestRegressor(n_estimators=700,
                               min_samples_split=5, random_state=96,
                               n_jobs=-1)  # Use maximum number of cores.
    # rf = make_pipeline(StandardScaler(), rf)
    rf.fit(x_training_RF, y_training)
    return rf

def GB_training(x_training_RF, y_training):
    reg1 = RandomForestRegressor(n_estimators=700,
                               min_samples_split=5, random_state=96,
                               n_jobs=-1)
    reg2 = BaggingRegressor(n_estimators=700)
    gb = reg1
    # gb = VotingRegressor(estimators=[('fg', reg1), ('bg', reg2)])
    gb.fit(x_training_RF, y_training)
    return gb

def spot_prediction(dict_df, training_type='Solar0'):
    
    training_length = 24*4*31 
    prediction_length = 24*4*31
    
    df_training, df_prediction, df_solar = split_training_dataframe(dict_df, training_type, training_length, prediction_length)
    x_training_RF, y_training = get_training_parameter(df_training, training_type)
    # RF_regression = RF_training(x_training_RF, y_training)
    RF_regression = GB_training(x_training_RF, y_training)
    x_prediction_RF, y_real = get_prediction_parameter(df_prediction, training_type)
    y_predict_RF = RF_regression.predict(x_prediction_RF)

    # nov_parameter = get_Nov_parameter(dict_df, training_type, prediction_length, occu = True)
    # y_predict_nov = RF_regression.predict(nov_parameter)
    # This if is to predict unseen data. Ask Nam personally if you are interested
    # if prediction_length == 0:
    #     prediction_length = training_length
    y_naive = df_solar[training_type].shift(2688)[-prediction_length:].values if training_type.find('Solar') != -1 else df_solar['consumption'].shift(2688)[-prediction_length:].values
    # return y_predict_RF, y_naive, y_real, y_predict_nov 
    return y_predict_RF, y_naive, y_real

In [3]:
def predict_all(dict_df):
    
    y_predict_RF = []
    y_naive = []
    y_real = []
    y_oct = []
    
    for key in dict_df:
        if any([i in key for i in ['Building','Solar']]):    
            print('Predicting for', key)
            
            y_RF_temp, y_naive_temp, y_real_temp, y_predict_oct = spot_prediction(dict_df, key)
            y_predict_RF.append(y_RF_temp)
            y_naive.append(y_naive_temp)
            y_real.append(y_real_temp)
            y_oct.append(y_predict_oct)
    return y_predict_RF, y_naive, y_real, y_oct

In [4]:
def Lab_training(x_training_RF, y_training, reg, estimators = 700):
    reg1 = RandomForestRegressor(n_estimators=estimators,
                               min_samples_split=5, random_state=96,
                               n_jobs=-1)
    reg2 = BaggingRegressor(n_estimators=estimators)
    reg3 = ExtraTreesRegressor(n_estimators=estimators)
    model = {'rf': reg1, 'br': reg2, 'et': reg3}
    reg_model = model[reg]
    # gb = VotingRegressor(estimators=[('fg', reg1), ('bg', reg2)])
    reg_model.fit(x_training_RF, y_training)
    return reg_model

## With the prediction tester above, we tried different classic techniques and found the best fits in Phase1. Some of testing results are recorded below:
- Gradient boosting: 0.796
- Extra Tree: 0.718 (0.793, 1.089, 0.864, 0.643, 0.735, 0.882, 0.614, 0.579, 0.609, 0.612, 0.598, 0.601)
- BaggingRegressor estimator 100: 0.706 (0.847, 1.065, 0.875, 0.644, 0.567, 0.887, 0.615, 0.574, 0.608, 0.605, 0.583, 0.602)
- RF estimator 700: 0.705 (0.841, 1.06, 0.875, 0.642, 0.56, 0.894, 0.613, 0.582, 0.607, 0.6, 0.584, 0.604)

In [5]:
def build_prediction(df, training_type, reg, estimators, occu = 'False'):
    training_length = 24*4*31
    df_training = df[training_type][-training_length:]
    dew_tempC_tr = df_training['dewpoint_temperature (degC)']
    tempC_tr = df_training['temperature (degC)']
    radiation_tr = df_training['surface_solar_radiation (W/m^2)']
    thermal_radiation_tr = df_training['surface_thermal_radiation (W/m^2)']
    cloudcover_tr = df_training['total_cloud_cover (0-1)']
    humidity_tr = df_training['relative_humidity ((0-1))']
    time_tr = pd.CategoricalIndex(df_training.index.time).codes.astype(float) 
    if training_type.find('Building') != -1:
        y_training_Sep = np.array(df_training['consumption'])
        weekday_tr = df_training['day_type'].astype('category').cat.codes.astype(float)
        lag_data_tr = np.array(df[training_type]['consumption'][-training_length*2:-training_length])
        occup_tr = df_training['occupancy (0-1)']
        x_training_Sep = np.column_stack((occup_tr, tempC_tr, humidity_tr, cloudcover_tr, radiation_tr, lag_data_tr, time_tr, weekday_tr)) if occu else np.column_stack((tempC_tr, humidity_tr, cloudcover_tr, radiation_tr, lag_data_tr, time_tr, weekday_tr))   
    if training_type.find('Solar') != -1:
        y_training_Sep = np.array(df_training[training_type])
        x_training_Sep = np.column_stack((cloudcover_tr, radiation_tr, time_tr)) 
    model = Lab_training(x_training_Sep, y_training_Sep, reg, estimators)
    oct_parameter = get_Oct_parameter(df, training_type, training_length, occu)
    y_predict_oct = model.predict(oct_parameter)
    return y_predict_oct

def predict_oct(dict_df, model_request):
    
    y_oct = []
    for key in dict_df:
        if any([i in key for i in ['Building','Solar']]):    
            print('Predicting for', key)
            y_predict_oct = build_prediction(dict_df, key, model_request[key]['reg'], model_request[key]['estimators'], model_request[key]['occupancy'])
            y_oct.append(y_predict_oct)
    return y_oct

In [6]:
model_request = {
    'Building0': {
        'reg': 'et',
        'estimators': 700,
        'occupancy': True
    }, 
    'Building1': {
        'reg': 'rf',
        'estimators': 700,
        'occupancy': False
    },
    'Building3': {
        'reg': 'et',
        'estimators': 700,
        'occupancy': False
    }, 
    'Building4': {
        'reg': 'br',
        'estimators': 1000,
        'occupancy': True
    },
    'Building5': {
        'reg': 'rf',
        'estimators': 700,
        'occupancy': False
    }, 
    'Building6': {
        'reg': 'br',
        'estimators': 700,
        'occupancy': True
    },
    'Solar0': {
        'reg': 'br',
        'estimators': 700,
        'occupancy': False
    }, 
    'Solar1': {
        'reg': 'br',
        'estimators': 100,
        'occupancy': False
    },
    'Solar2': {
        'reg': 'br',
        'estimators': 100,
        'occupancy': False
    }, 
    'Solar3': {
        'reg': 'br',
        'estimators': 700,
        'occupancy': False
    },
    'Solar4': {
        'reg': 'br',
        'estimators': 100,
        'occupancy': False
    }, 
    'Solar5': {
        'reg': 'et',
        'estimators': 700,
        'occupancy': False}
}

In [ ]:
predict_oct = predict_oct(data_cleanned_v2, model_request)

## For Solar data, we found the solar capacity increasemnet of Solar3 from May 2020 (comparing Oct 2019 to Oct 2020, and also for the trends over the year)

In [ ]:
check_loads_plot(data_cleanned_v2, 'Solar0')

## Compared to Solar2 and Solar4, the Solar3 is irregular. 
- We considered either to scalled up the consumption, omit the abnormal periods, or using real pattern to patch the abnormal periods.
- We ended up using the Solar 2 data for compensation.
- This is a pure assumption based compensation hence we didn't include it as data preprocessing as the other choices might give better results

In [7]:
check_loads_plot(data_cleanned_v2, 'Solar2')

In [ ]:
target = 'Solar5'
fig = go.Figure()
time = data_cleanned[target]['consumption'].index if target.find('Building') != -1 else data_cleanned[target][target].index[-24*4*31:] 
# real_value = data_cleanned[user_type][user_type][predict_range[0]:]
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_cnn, line_width = 0.8, opacity = 1, name = 'Prediction- M-CNN'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_rf, line_width = 0.8, opacity = 1, name = 'Prediction- RF'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_real, line_width = 0.8, opacity = 1, name = 'Real'
        )
    )
fig.layout.xaxis.title = "Time"
fig.layout.yaxis.title = "ELectricity (kW)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
##, tickangle = 45 
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

In [ ]:
check_loads_plot(data_cleanned_v2, 'Solar4')

In [ ]:
datetime = pd.to_datetime('2020-05-20 00:00:00')
cut_time = find_index_v2(data_cleanned_v2, 'Solar2', datetime)
cut_time

In [ ]:

data_cleanned_v2['Solar3']['Solar3'][:cut_time] = data_cleanned_v2['Solar2']['Solar2'][:cut_time]
data_cleanned_v2['Solar5']['Solar5'] =  data_cleanned_v2['Solar5']['Solar5'][96*300:]
data_cleanned_v2['Solar0']['Solar0'] =  data_cleanned_v2['Solar0']['Solar0'][96*120:]

In [7]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[]

In [7]:
tf.__version__

'2.7.0'

In [19]:
LOSS_THRESHOLD = 0.8
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('val_loss') < LOSS_THRESHOLD):   
            print("\nReached %2.2f%% validation loss, so stopping training!!" %(LOSS_THRESHOLD*100))   
            self.model.stop_training = True
callbacks = myCallback()

## Calibrations on RM solar prediction model (RF is the controlled group)
### Repalcing CNN with ResNet is an unverified improvement (since we didn't use it in Phase1 when we had better predicting performance)
- The intuition is to leverage [Deep Double Descent ](https://openai.com/blog/deep-double-descent/) and [Over-parameterization](https://openreview.net/forum?id=C3qvk5IQIJY)
- We didn't go with ResNet 101 since it takes too long to run, but we left it customisable by calling NN_model.ResNet, e.g. NN_model.ResNet([5, 8, 36, 3]) for ResNet152

### Some tested results are listed as below (this block can be used for customised setting's observations):

- Solar1 and Solar4 got better result using CNN, solar 0235 got better results using resnet
- Solar0's best block_layers = [3, 4, 23, 3]
- Solar 1 - 0.11 0.103, 0.23 loss -> 0.1-> 0.66.
- Solar 3 - 0.9575930659246576 - 0.2373 - 0.21 - 0.228 - 0.08
- Solar 4 : 0.5321199592458181 - 0.12, 014 , 0.11 without temperature
- S0 - 0.038  0.042 (early stop)    0.245
- Solar0:   1211 - mase loss 0.6 MSE 0.12 -> 0.77    12 - mase loss 0.77 MSE 0.14 -> 0.769  res101 0.69 MSE 0.11 ->0.731 res152
- Solar1:  3463 - loss 0.07 MSE 0.095 -> 0.66 
- solar2 2222 -> 0.16 -> 0.63
- solar4 2222 -> loss 0.07 0.11 ->0.66    early stop100 loss 0.19 -> 0.20 -> 0.68 stop200 0.65
- Solar5 2222 -> loss 1.55 ->0.11 -> 0.627 -> 0.5607581560999154


## Causality test:
- temperature (degC) is a cause of Solar0
- dewpoint_temperature (degC) is a cause of Solar0
- wind_speed (m/s) is a cause of Solar0
- relative_humidity ((0-1)) is a cause of Solar0
- surface_solar_radiation (W/m^2) is a cause of Solar0


In [6]:
def RM_prediction(data_cleanned, user_type, NN_type):
## motifs discovery range
    test_days = 31
    train_days = int(len(data_cleanned[user_type])/96-test_days)
    # motifs_range = [-24*4*31,0] 
    train_range = [-24*4*(train_days+test_days),-24*4*test_days] 
    motifs_range = train_range
    test_range = [-24*4*test_days, 0]
    RM = RM_model(data_cleanned)
    motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif = RM.motifs_discovery_v2(user_type, motifs_range)
    ### training data
    train_data = data_cleanned[user_type][train_range[0]:].copy() if train_range[1] ==0 else data_cleanned[user_type][train_range[0]:train_range[1]].copy()
    y_train = train_data[user_type].values.reshape(train_days, 96) if user_type.find('Solar') != -1 else train_data['consumption'].values.reshape(train_days, 96)
    cloudcover_train = train_data ['total_cloud_cover (0-1)'].values.reshape(train_days, 96)
    humidity_train = train_data ['relative_humidity ((0-1))'].values.reshape(train_days, 96)
    radiation_train = train_data ['surface_solar_radiation (W/m^2)'].values.reshape(train_days, 96)
    temperature_train = train_data['temperature (degC)'].values.reshape(train_days, 96)
    sealevel_train = train_data['mean_sea_level_pressure (Pa)'].values.reshape(train_days, 96)
    wind_train = train_data['wind_speed (m/s)'].values.reshape(train_days, 96)
    temperature_train = [i-temperature_motif.values for i in temperature_train] 
    sealevel_train = [i-sealevel_motif.values for i in sealevel_train]
    wind_train = [i-wind_motif.values for i in wind_train]
    cloudcover_train = [i-cloudcover_motif.values for i in cloudcover_train] 
    humidity_train = [i-humidity_motif.values for i in humidity_train]
    radiation_train = [i-radiation_motif.values for i in radiation_train]
    times_train = pd.CategoricalIndex(train_data.index.time).codes.astype(float)
    sun_cycle_train = np.maximum(np.sin(2*np.pi *(times_train+16)/96), 0 ).reshape(train_days, 96)
    times_train = times_train.reshape(train_days, 96)
    weekday_train = pd.CategoricalIndex(train_data.index.weekday).codes.astype(float).reshape(train_days, 96)
    motifs_train = [motif_pattern for i in range(train_days)]
    motifs_test = [motif_pattern for i in range(test_days)]
    motifs_train = np.array(motifs_train)
    motifs_test = np.array(motifs_test)
    month_train = pd.CategoricalIndex(train_data.index.month).codes.astype(float).reshape(train_days, 96)
    # year_cycle_train = 1+np.sin(2*np.pi *(month_train-8)/12)
    year_cycle_train = abs(np.sin(2*np.pi*(month_train-5)/24))
    year_cycle_train = year_cycle_train.reshape(train_days, 96)
    weekend_train = np.where((weekday_train == 5) | (weekday_train == 6), True, False)
    if user_type.find('Solar') != -1:
        if user_type = 'Solar0':
            year_cycle_train = abs(np.sin(2*np.pi*(month_train-1)/24))
        x_train = np.column_stack((motifs_train, cloudcover_train, radiation_train, times_train, year_cycle_train)) 
        # x_train_cnn = np.dstack((motifs_train, temperature_train , cloudcover_train, radiation_train, year_cycle_train))
        x_train_cnn = np.dstack((motifs_train, humidity_train, temperature_train , cloudcover_train,wind_train,  radiation_train, year_cycle_train))
    else:
        occupancy_motif = motif_data['occupancy (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
        occupancy_train = train_data['occupancy (0-1)'].values.reshape(train_days, 96)
        occupancy_train = [i-occupancy_motif.values for i in occupancy_train] 
        x_train = np.column_stack((motifs_train, temperature_train, occupancy_train, humidity_train, cloudcover_train, radiation_train, weekday_train, times_train)) 
        x_train_cnn = np.dstack((motifs_train, weekend_train, temperature_train, radiation_train, month_train))
    Solar_pre_model = NN_model(x_train_cnn, y_train, 96) 
    if NN_type == 'CNN':
        model_cnn = Solar_pre_model.CNN_series(error_matric= 'mae')
    elif NN_type == 'ResNet':
        model_cnn = Solar_pre_model.ResNet(error_matric= 'mae', block_layers = [2, 2, 2, 2])
    elif NN_type == 'Transformer':
        model_cnn = Solar_pre_model.Transformer(
            head_size = 256,
            num_heads = 4,
            ff_dim = 4,
            num_transformer_blocks = 2,
            mlp_units = [128],
            dropout=0.3,
            mlp_dropout=0.25,
            error_matric = 'mae')
    else:
        print('Error: Model does not exist!')
        return False
    ### testing data
    test_data = data_cleanned[user_type][test_range[0]:].copy()
    y_test = test_data[user_type].values.reshape(test_days, 96) if user_type.find('Solar') != -1 else test_data['consumption'].values.reshape(test_days, 96)
    y_max_test = [[np.max(i), np.min(i)] for i in y_test]
    y_max_test = np.array(y_max_test)
    cloudcover_test = test_data ['total_cloud_cover (0-1)'].values.reshape(test_days, 96)
    humidity_test = test_data ['relative_humidity ((0-1))'].values.reshape(test_days, 96)
    radiation_test = test_data ['surface_solar_radiation (W/m^2)'].values.reshape(test_days, 96)
    temperature_test = test_data['temperature (degC)'].values.reshape(test_days, 96)
    sealevel_test = test_data['mean_sea_level_pressure (Pa)'].values.reshape(test_days, 96)
    wind_test = test_data['wind_speed (m/s)'].values.reshape(test_days, 96)
    temperature_test = [i-temperature_motif.values for i in temperature_test] 
    cloudcover_test = [i-cloudcover_motif.values for i in cloudcover_test] 
    humidity_test = [i-humidity_motif.values for i in humidity_test]
    radiation_test = [i-radiation_motif.values for i in radiation_test]
    sealevel_test = [i-sealevel_motif.values for i in sealevel_test]
    wind_test = [i-wind_motif.values for i in wind_test]
    times_test = pd.CategoricalIndex(test_data.index.time).codes.astype(float)
    sun_cycle_test = np.maximum(np.sin(2*np.pi *(times_test+16)/96), 0 ).reshape(test_days, 96)
    times_test = times_test.reshape(test_days, 96)
    weekday_test = pd.CategoricalIndex(test_data.index.weekday).codes.astype(float).reshape(test_days, 96)
    month_test = pd.CategoricalIndex(test_data.index.month).codes.astype(float).reshape(test_days, 96)
    month_test = month_test+month_train[-1][-1]+1
    year_cycle_test = abs(np.sin(2*np.pi*(month_test-5)/24))
    year_cycle_test = year_cycle_test.reshape(test_days, 96)
    weekend_test = np.where((weekday_test == 5) | (weekday_test == 6), True, False)
    if user_type.find('Solar') != -1:
        if user_type = 'Solar0':
            year_cycle_test = abs(np.sin(2*np.pi*(month_test-1)/24))
        x_test = np.column_stack((motifs_test, cloudcover_test, radiation_test, times_test, month_test)) 
        # x_test_cnn = np.dstack((motifs_test, temperature_test, cloudcover_test, radiation_test, year_cycle_test)) 
        x_test_cnn = np.dstack((motifs_test, humidity_test, temperature_test, cloudcover_test, wind_test, radiation_test, year_cycle_test))
    else:
        occupancy_test = test_data['occupancy (0-1)'].values.reshape(test_days, 96)
        occupancy_test = [i-occupancy_motif.values for i in occupancy_test] 
        x_test = np.column_stack((motifs_test, temperature_test, occupancy_test, humidity_test, cloudcover_test, radiation_test, weekday_test, times_test)) 
        x_test_cnn = np.dstack((motifs_test, weekend_test, temperature_test, radiation_test, month_test))
    ### 
    test_epochs = 2000
    test_patience = 100
    # y_predict_max = model_cnn_max.predict(x_test_cnn).reshape(test_days,2)
    model_cnn.fit(x_train_cnn, y_train, epochs = test_epochs, validation_data = (x_test_cnn, y_test), batch_size=24, verbose = 1, callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=test_patience)])
    # model_cnn.fit(x_train_cnn, y_train, epochs = test_epochs, validation_data = (x_test_cnn, y_test), batch_size=24, verbose = 1, callbacks=[callbacks])
    y_predict_RF, y_naive, y_real = spot_prediction(data_cleanned, user_type)
    y_predict_cnn = model_cnn.predict(x_test_cnn).reshape(test_days*96)
    # y_predict_cnn = model_res.predict(x_test_cnn).reshape(test_days*96)
    sust = np.mean(np.abs(y_naive - y_real))
    diff_RF = np.mean(np.abs(y_real - y_predict_RF))
    diff_CNN = np.mean(np.abs(y_real - y_predict_cnn))
    print('RF accuracy: ', diff_RF/sust)
    print('Motif '+NN_type+' accuracy: ', diff_CNN/sust)
    return model_cnn, y_predict_cnn, y_real, y_predict_RF



In [22]:
model_solar, y_cnn, y_real, y_rf = RM_prediction(data_cleanned_v2, 'Solar1', 'Transformer')
# solar_list = ['Solar0', 'Solar1', 'Solar2', 'Solar3', 'Solar4', 'Solar5']
# for i in solar_list:
#     print(i+' prediction:')
#     model_solar, x_oct, y_cnn, y_rf, y_real= RM_prediction(data_cleanned_v2, i, 'CNN')
#     model_solar, x_oct, y_cnn, y_rf, y_real= RM_prediction(data_cleanned_v2, i, 'ResNet')


Refined Motif at:  2020-02-28 00:00:00
Epoch 1/2000
14/14 [==============================] - 5s 256ms/step - loss: 6.8985 - val_loss: 5.2219
Epoch 2/2000
14/14 [==============================] - 3s 231ms/step - loss: 6.1836 - val_loss: 4.7647
Epoch 3/2000
14/14 [==============================] - 3s 229ms/step - loss: 5.4405 - val_loss: 4.3663
Epoch 4/2000
14/14 [==============================] - 3s 225ms/step - loss: 4.9357 - val_loss: 4.0125
Epoch 5/2000
14/14 [==============================] - 3s 234ms/step - loss: 4.3972 - val_loss: 3.7046
Epoch 6/2000
14/14 [==============================] - 3s 229ms/step - loss: 4.0598 - val_loss: 3.4340
Epoch 7/2000
14/14 [==============================] - 3s 234ms/step - loss: 3.7529 - val_loss: 3.2038
Epoch 8/2000
14/14 [==============================] - 3s 232ms/step - loss: 3.4271 - val_loss: 3.0017
Epoch 9/2000
14/14 [==============================] - 3s 227ms/step - loss: 3.1912 - val_loss: 2.8214
Epoch 10/2000
14/14 [======================

In [27]:
from numba import jit

@jit
def weighted_DTW(series_1, series_2, weight):
    #### Series_1 and Series_2 are time series list, weight is weighted matrix list with two dimention
    #### default weight can be np.ones(len(series_1))
    #### weight stands for different attentions on the sub-patterns
    l1 = len(series_1)
    l2 = len(series_2)
    cum_sum = np.full((l1 + 1, l2 + 1), np.inf)
    cum_sum[0, 0] = 0.
    for i in range(l1):
        for j in range(l2):
            diff = (series_1[i] - series_2[j])
            distance = diff*diff*weight[i][j]
            cum_sum[i + 1, j + 1] = distance
            cum_sum[i + 1, j + 1] += min(cum_sum[i, j + 1],
                                            cum_sum[i + 1, j],
                                            cum_sum[i, j])
    acc_cost_mat = cum_sum[1:, 1:]
    return np.sqrt(acc_cost_mat[-1][-1])
class RM_model:
    """A class for representative daily data discovery"""

    ### fixed threshold
    def motif_dtw_24_w(self, time_series, m, threshold, weight):
        weighted_matrix = np.ones((len(weight), len(weight)))
        for i in range(len(weight)):
            for j in range(len(weight)):
                weighted_matrix[i][j] = max(weight[i], weight[j])
        # find the motif with most sub-patterns near than threshold
        length = len(time_series)-m
        sliding = int(length/m)
        distance_profile = []
        average_profile = []
        motif_co = 0
        time_series = np.array(time_series)
        for i in range(sliding+1):
            pattern = time_series[i*m:i*m+m]
            counter = 0
            average_profile.append(0)
            for j in range(sliding+1):
                distance = weighted_DTW(pattern, time_series[j*m:j*m+m], weighted_matrix)
                average_profile[i] = average_profile[i]+distance
                if distance < threshold:
                    counter+=1
            average_profile[i] = average_profile[i]/sliding
            distance_profile.append(counter - average_profile[i]*0.001)
        motif_co = np.argsort(distance_profile)[-1]
        motif_se = np.argsort(distance_profile)[-2]
        motif = dict (
            distance_profile = distance_profile,
            motif_position = motif_co*m,
            motif_sec = motif_se*m,
            average_profile = average_profile
        )

    ### Adoptable threshold
    def motif_dtw_24_w_dy(self, time_series, m, weight):
        weighted_matrix = np.ones((len(weight), len(weight)))
        for i in range(len(weight)):
            for j in range(len(weight)):
                weighted_matrix[i][j] = max(weight[i], weight[j])
        # find the motif with most sub-patterns near than threshold
        length = len(time_series)-m
        sliding = int(length/m)
        distance_stack = []
        distance_profile = []
        average_profile = []
        motif_co = 0
        # change to array for numba accelerataion
        time_series = np.array(time_series)
        for i in range(sliding+1):
            pattern = time_series[i*m:i*m+m]
            average_profile.append(0)
            for j in range(sliding+1):
                distance = weighted_DTW(pattern, time_series[j*m:j*m+m], weighted_matrix)
                average_profile[i] = average_profile[i]+distance
                distance_stack.append(distance)
            average_profile[i] = average_profile[i]/sliding
        threshold = np.median(distance_stack)
        # print('Distance matrix: ',distance_stack)
        for k in range(sliding+1):  
            pattern = time_series[i*m:i*m+m]  
            counter = 0
            for l in range(sliding+1):
                distance = distance_stack[k*(sliding+1)+l]
                if distance < threshold:
                    counter+=1
            distance_profile.append(counter - average_profile[k]*0.001)
        motif_co = np.argsort(distance_profile)[-1]
        motif_se = np.argsort(distance_profile)[-2]
        motif = dict (
            distance_profile = distance_profile,
            motif_position = motif_co*m,
            motif_sec = motif_se*m,
            average_profile = average_profile
        )
        return motif

    def __init__(self, data_cleanned):
         self.data_cleanned = data_cleanned

    def motifs_discovery_v2(self, user_type, motifs_range):
        motif_data =  self.data_cleanned[user_type][motifs_range[0]:] if motifs_range[1] ==0 else self.data_cleanned[user_type][motifs_range[0]:motifs_range[1]]
        targeted_series = motif_data[user_type] if user_type.find('Solar') != -1 else motif_data['consumption']
        ## No special attention applied
        motifs = self.motif_dtw_24_w_dy(targeted_series, 96, np.ones((96)))
        motif_pattern = targeted_series[motifs['motif_position']:motifs['motif_position']+96]
        print('Refined Motif at: ',motif_data.index[motifs['motif_position']])
        temperature_motif = motif_data['temperature (degC)'][motifs['motif_position']:motifs['motif_position']+96]
        cloudcover_motif = motif_data ['total_cloud_cover (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
        humidity_motif = motif_data ['relative_humidity ((0-1))'][motifs['motif_position']:motifs['motif_position']+96]
        radiation_motif = motif_data ['surface_solar_radiation (W/m^2)'][motifs['motif_position']:motifs['motif_position']+96]
        sealevel_motif = motif_data ['mean_sea_level_pressure (Pa)'][motifs['motif_position']:motifs['motif_position']+96]
        wind_motif = motif_data ['wind_speed (m/s)'][motifs['motif_position']:motifs['motif_position']+96]
        times_motif = pd.CategoricalIndex(motif_data.index.time).codes.astype(float) 
        return motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif

    def motifs_discovery(self, user_type, motifs_range):
        motif_data =  self.data_cleanned[user_type][motifs_range[0]:] if motifs_range[1] ==0 else self.data_cleanned[user_type][motifs_range[0]:motifs_range[1]]
        targeted_series = motif_data[user_type] if user_type.find('Solar') != -1 else motif_data['consumption']
        ## No special attention applied
        motifs = self.motif_dtw_24_w_dy(targeted_series, 96, np.ones((96)))
        motif_pattern = targeted_series[motifs['motif_position']:motifs['motif_position']+96]
        print('Refined Motif at: ',motif_data.index[motifs['motif_position']])
        temperature_motif = motif_data['temperature (degC)'][motifs['motif_position']:motifs['motif_position']+96]
        cloudcover_motif = motif_data ['total_cloud_cover (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
        humidity_motif = motif_data ['relative_humidity ((0-1))'][motifs['motif_position']:motifs['motif_position']+96]
        radiation_motif = motif_data ['surface_solar_radiation (W/m^2)'][motifs['motif_position']:motifs['motif_position']+96]
        times_motif = pd.CategoricalIndex(motif_data.index.time).codes.astype(float) 
        return motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, times_motif
    
    def motifs_discovery_v3(self, user_type, motifs_range):
        motif_data =  self.data_cleanned[user_type][motifs_range[0]:] if motifs_range[1] ==0 else self.data_cleanned[user_type][motifs_range[0]:motifs_range[1]]
        targeted_series = motif_data["demand(kWh)/production(kW)"] if user_type.find('Solar') != -1 else motif_data['consumption']
        ## No special attention applied
        motifs = self.motif_dtw_24_w_dy(targeted_series, 96, np.ones((96)))
        motif_pattern = targeted_series[motifs['motif_position']:motifs['motif_position']+96]
        print('Refined Motif at: ',motif_data.index[motifs['motif_position']])
        temperature_motif = motif_data['temperature (degC)'][motifs['motif_position']:motifs['motif_position']+96]
        cloudcover_motif = motif_data ['total_cloud_cover (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
        humidity_motif = motif_data ['relative_humidity ((0-1))'][motifs['motif_position']:motifs['motif_position']+96]
        radiation_motif = motif_data ['surface_solar_radiation (W/m^2)'][motifs['motif_position']:motifs['motif_position']+96]
        sealevel_motif = motif_data ['mean_sea_level_pressure (Pa)'][motifs['motif_position']:motifs['motif_position']+96]
        wind_motif = motif_data ['wind_speed (m/s)'][motifs['motif_position']:motifs['motif_position']+96]
        times_motif = pd.CategoricalIndex(motif_data.index.time).codes.astype(float) 
        return motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif

In [28]:
y_predict_RF, y_naive, y_real = spot_prediction(data_cleanned_v2, 'Solar1')

In [57]:
user_type = 'Solar1'
test_days = 30
train_days = int(len(data_cleanned[user_type])/96-test_days)
data_cleanned = data_cleanned_v3
train_range = [-24*4*(train_days+test_days),-24*4*test_days] 
motifs_range = train_range
test_range = [-24*4*test_days, 0]
RM = RM_model(data_cleanned)
motifs, motif_data, motif_pattern, temperature_motif, cloudcover_motif, humidity_motif, radiation_motif, sealevel_motif, wind_motif, times_motif = RM.motifs_discovery_v3(user_type, motifs_range)
# motifs_range = [-24*4*31,0] 
train_range = [-24*4*(train_days+test_days),-24*4*test_days] 
train_data = data_cleanned[user_type][train_range[0]:].copy() if train_range[1] ==0 else data_cleanned[user_type][train_range[0]:train_range[1]].copy()
y_train = train_data["demand(kWh)/production(kW)"].values.reshape(train_days, 96) if user_type.find('Solar') != -1 else train_data['consumption'].values.reshape(train_days, 96)
cloudcover_train = train_data ['total_cloud_cover (0-1)'].values.reshape(train_days, 96)
humidity_train = train_data ['relative_humidity ((0-1))'].values.reshape(train_days, 96)
radiation_train = train_data ['surface_solar_radiation (W/m^2)'].values.reshape(train_days, 96)
temperature_train = train_data['temperature (degC)'].values.reshape(train_days, 96)
sealevel_train = train_data['mean_sea_level_pressure (Pa)'].values.reshape(train_days, 96)
wind_train = train_data['wind_speed (m/s)'].values.reshape(train_days, 96)
temperature_train = [i-temperature_motif.values for i in temperature_train] 
sealevel_train = [i-sealevel_motif.values for i in sealevel_train]
wind_train = [i-wind_motif.values for i in wind_train]
cloudcover_train = [i-cloudcover_motif.values for i in cloudcover_train] 
humidity_train = [i-humidity_motif.values for i in humidity_train]
radiation_train = [i-radiation_motif.values for i in radiation_train]
times_train = pd.CategoricalIndex(train_data.index.time).codes.astype(float)
sun_cycle_train = np.maximum(np.sin(2*np.pi *(times_train+16)/96), 0 ).reshape(train_days, 96)
times_train = times_train.reshape(train_days, 96)
weekday_train = pd.CategoricalIndex(train_data.index.weekday).codes.astype(float).reshape(train_days, 96)
motifs_train = [motif_pattern for i in range(train_days)]
motifs_test = [motif_pattern for i in range(test_days)]
motifs_train = np.array(motifs_train)
motifs_test = np.array(motifs_test)
month_train = pd.CategoricalIndex(train_data.index.month).codes.astype(float).reshape(train_days, 96)
# year_cycle_train = 1+np.sin(2*np.pi *(month_train-8)/12)
year_cycle_train = abs(np.sin(2*np.pi*(month_train-5)/24))
year_cycle_train = year_cycle_train.reshape(train_days, 96)
weekend_train = np.where((weekday_train == 5) | (weekday_train == 6), True, False)
if user_type.find('Solar') != -1:
    x_train = np.column_stack((motifs_train, cloudcover_train, radiation_train, times_train, year_cycle_train)) 
    # x_train_cnn = np.dstack((motifs_train, temperature_train , cloudcover_train, radiation_train, year_cycle_train))
    x_train_cnn = np.dstack((motifs_train, humidity_train, temperature_train , cloudcover_train,wind_train,  radiation_train, year_cycle_train))
else:
    occupancy_motif = motif_data['occupancy (0-1)'][motifs['motif_position']:motifs['motif_position']+96]
    occupancy_train = train_data['occupancy (0-1)'].values.reshape(train_days, 96)
    occupancy_train = [i-occupancy_motif.values for i in occupancy_train] 
    x_train = np.column_stack((motifs_train, temperature_train, occupancy_train, humidity_train, cloudcover_train, radiation_train, weekday_train, times_train)) 
    x_train_cnn = np.dstack((motifs_train, weekend_train, temperature_train, radiation_train, month_train))
### Test data
test_data = data_cleanned[user_type][test_range[0]:].copy()
y_test = test_data['demand(kWh)/production(kW)'].values.reshape(test_days, 96) if user_type.find('Solar') != -1 else test_data['consumption'].values.reshape(test_days, 96)
y_max_test = [[np.max(i), np.min(i)] for i in y_test]
y_max_test = np.array(y_max_test)
cloudcover_test = test_data ['total_cloud_cover (0-1)'].values.reshape(test_days, 96)
humidity_test = test_data ['relative_humidity ((0-1))'].values.reshape(test_days, 96)
radiation_test = test_data ['surface_solar_radiation (W/m^2)'].values.reshape(test_days, 96)
temperature_test = test_data['temperature (degC)'].values.reshape(test_days, 96)
sealevel_test = test_data['mean_sea_level_pressure (Pa)'].values.reshape(test_days, 96)
wind_test = test_data['wind_speed (m/s)'].values.reshape(test_days, 96)
temperature_test = [i-temperature_motif.values for i in temperature_test] 
cloudcover_test = [i-cloudcover_motif.values for i in cloudcover_test] 
humidity_test = [i-humidity_motif.values for i in humidity_test]
radiation_test = [i-radiation_motif.values for i in radiation_test]
sealevel_test = [i-sealevel_motif.values for i in sealevel_test]
wind_test = [i-wind_motif.values for i in wind_test]
times_test = pd.CategoricalIndex(test_data.index.time).codes.astype(float)
sun_cycle_test = np.maximum(np.sin(2*np.pi *(times_test+16)/96), 0 ).reshape(test_days, 96)
times_test = times_test.reshape(test_days, 96)
weekday_test = pd.CategoricalIndex(test_data.index.weekday).codes.astype(float).reshape(test_days, 96)
month_test = pd.CategoricalIndex(test_data.index.month).codes.astype(float).reshape(test_days, 96)
year_cycle_test = abs(np.sin(2*np.pi*(month_test-5)/24))
year_cycle_test = year_cycle_test.reshape(test_days, 96)
weekend_test = np.where((weekday_test == 5) | (weekday_test == 6), True, False)
x_test = np.column_stack((motifs_test, cloudcover_test, radiation_test, times_test, month_test)) 
# x_test_cnn = np.dstack((motifs_test, temperature_test, cloudcover_test, radiation_test, year_cycle_test)) 
x_test_cnn = np.dstack((motifs_test, humidity_test, temperature_test, cloudcover_test, wind_test, radiation_test, year_cycle_test))
x_train_rf = x_train_cnn.reshape(train_days*96, 7)
x_test_rf = x_test_cnn.reshape(test_days*96, 7)
RF_regression = GB_training(x_train_rf, y_train.reshape(-1))

Refined Motif at:  2020-05-28 00:00:00


In [60]:
prediction_length = 30*96
y_naive = data_cleanned[user_type]['demand(kWh)/production(kW)'].shift(30*96)[-prediction_length:].values
y_real = y_test.reshape(-1)
y_predict_RF = RF_regression.predict(x_test_rf)
sust = np.mean(np.abs(y_naive - y_real))
diff_RF = np.mean(np.abs(y_real - y_predict_RF))
print('RF MASE: ', diff_RF/sust)

RF accuracy:  0.5810726115497666


In [10]:
data_cleanned_v3['Solar1']

,demand(kWh)/production(kW),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
2018-12-31 13:00:00,0.00,16.32,12.40,3.13,101292.86,0.78,0.00,330.62,0.03
2018-12-31 13:15:00,0.00,16.32,12.40,3.13,101292.86,0.78,0.00,330.62,0.03
2018-12-31 13:30:00,0.00,16.32,12.40,3.13,101292.86,0.78,0.00,330.62,0.03
2018-12-31 13:45:00,0.00,16.32,12.40,3.13,101292.86,0.78,0.00,330.62,0.03
2018-12-31 14:00:00,0.00,15.54,12.50,2.72,101256.78,0.82,0.00,326.49,0.07
...,...,...,...,...,...,...,...,...,...
2020-11-30 22:45:00,2.32,23.56,14.05,6.34,100293.63,0.55,268.04,365.36,0.93
2020-11-30 23:00:00,2.21,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:15:00,2.00,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:30:00,1.78,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99


In [18]:
model_solar.save('temp/model')

INFO:tensorflow:Assets written to: temp/model\assets


INFO:tensorflow:Assets written to: temp/model\assets


In [ ]:
target = 'Solar5'
fig = go.Figure()
time_sep = data_cleanned[target]['consumption'].index[-24*4*31:] if target.find('Building') != -1 else data_cleanned[target][target].index[-24*4*31:] 
# real_value = data_cleanned[user_type][user_type][predict_range[0]:]
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_cnn, line_width = 0.8, opacity = 1, name = 'Prediction- M-CNN'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_rf, line_width = 0.8, opacity = 1, name = 'Prediction- RF'
        )
    )
fig.add_trace(
        go.Scatter(
            x= time_sep, y=y_real, line_width = 0.8, opacity = 1, name = 'Real'
        )
    )
fig.layout.xaxis.title = "Time"
fig.layout.yaxis.title = "ELectricity (kW)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False)
##, tickangle = 45 
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

In [ ]:
check_loads_plot(data_cleanned_v2, 'Solar2')

In [85]:
# model for building consumptions (didn't provide better results than RF since buildings consumption has high variences that cannot be guided with one day's representative data)

def CNN_series_buildings(x_train_cnn, y_train):
    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(x_train_cnn.shape[1], x_train_cnn.shape[2])))
    model.add(Conv1D(80,10, strides=1, activation='relu',padding='causal'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(y_train.shape[1], activation='relu')) 
    model.compile(optimizer='adam', loss='mse')
    return model

## Evaluations for performances tracking
- The evaluation function below is a simplified version for tracking accuracy changes, which gives larger values than we seen on leaderbored since the length of shifted data are different to the evaluation function in competition.
- It works the same on tracking the accuracy changes
- For instance, comparing our first submission in Phase1, i.e., Sep22 to the last in October: 

In [ ]:
data_submitted = read_pickle('data\submissions_phase1.pickle')
predictions = data_submitted['submission_Oct9']
ini_predictions = data_submitted['submission_Sep22']

In [ ]:
total_MASE = 0
total_MASE_init = 0
data_processed = data_cleanned_v2
init_MSE = []
MSE = []
naive_shift = 96*28
for check_key in predictions.keys():
    oct_length = len(predictions[check_key])   
    real_value = data_processed[check_key][check_key][-oct_length:] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length:]
    naive_value = data_processed[check_key][check_key][-oct_length-naive_shift:-naive_shift] if check_key.find('Solar') != -1 else data_processed[check_key]['consumption'][-oct_length-naive_shift:-naive_shift]
    predictted_value = predictions[check_key]
    real_value = real_value.values
    naive_value = naive_value.values
    predictted_value = predictted_value.values
    ini_predictions_value = ini_predictions[check_key].values
    if check_key.find('Building') != -1:
        validation = data_processed[check_key]['missing_value'][-oct_length:]
        validation = np.invert(np.array(validation, dtype=bool))
        real_value = real_value[validation]
        naive_value = naive_value[validation]
        predictted_value = predictted_value[validation]
        ini_predictions_value = ini_predictions_value[validation]
        # print(len(naive_value), len(real_value), len(predictted_value))
    sust = np.mean(np.abs(naive_value - real_value))
    diff = np.mean(np.abs(real_value - predictted_value))
    init_diff = np.mean(np.abs(real_value - ini_predictions_value))
    mase = diff/sust
    init_mase = init_diff/sust
    MSE.append(diff)
    init_MSE.append(init_diff)
    total_MASE += mase
    total_MASE_init += init_mase
    print(check_key+' MASE: ' ,diff/sust)
    print(check_key+' initial MASE: ' ,init_mase)
print('average: ', total_MASE/12)
print('average (init): ', total_MASE_init/12)

## G-causality test

In [8]:
from statsmodels.tsa.stattools import grangercausalitytests

In [8]:
data_cleanned_v2['Solar0']

,Solar0,temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
2020-05-22 00:00:00,3.75,11.09,8.20,2.33,102140.01,0.82,174.42,309.67,0.79
2020-05-22 00:15:00,4.17,11.09,8.20,2.33,102140.01,0.82,174.42,309.67,0.79
2020-05-22 00:30:00,4.58,11.09,8.20,2.33,102140.01,0.82,174.42,309.67,0.79
2020-05-22 00:45:00,10.25,11.09,8.20,2.33,102140.01,0.82,174.42,309.67,0.79
2020-05-22 01:00:00,15.92,11.66,7.87,2.81,102115.71,0.78,337.41,291.93,0.67
...,...,...,...,...,...,...,...,...,...
2020-10-31 22:45:00,18.53,13.44,9.07,2.46,102352.43,0.75,278.75,333.88,0.99
2020-10-31 23:00:00,22.23,13.97,9.15,2.71,102386.08,0.73,394.75,353.90,0.98
2020-10-31 23:15:00,23.39,13.97,9.15,2.71,102386.08,0.73,394.75,353.90,0.98
2020-10-31 23:30:00,24.56,13.97,9.15,2.71,102386.08,0.73,394.75,353.90,0.98


In [13]:
grangercausalitytests(data_cleanned_v2['Solar2'][['Solar2', 'total_cloud_cover (0-1)']], maxlag=[1], verbose=True)
# a[5][0]['params_ftest'][1]


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=14.2096 , p=0.0002  , df_denom=33596, df_num=1
ssr based chi2 test:   chi2=14.2109 , p=0.0002  , df=1
likelihood ratio test: chi2=14.2079 , p=0.0002  , df=1
parameter F test:         F=14.2096 , p=0.0002  , df_denom=33596, df_num=1


{1: ({'ssr_ftest': (14.20959791157356, 0.00016381395244067986, 33596.0, 1),
   'ssr_chi2test': (14.210866776728183, 0.00016342402845380402, 1),
   'lrtest': (14.207862344876048, 0.00016368516089703357, 1),
   'params_ftest': (14.209597911540873, 0.00016381395244384527, 33596.0, 1.0)},
   array([[0., 1., 0.]])])}

In [ ]:
G_causality

In [19]:
# SOLAR_LIST = ['Solar'+str(i) for i in range(6)]
SOLAR_LIST = ['Solar3']
CHECK_INPUTS = data_cleanned_v2['Solar1'].keys()[1:]
for i in SOLAR_LIST:
    for j in CHECK_INPUTS:
        G_causality = grangercausalitytests(data_cleanned_v2[i][[i, j]], maxlag=[1], verbose=False)
        print("F test for "+j)
        print("F="+ "{:.4f}".format(G_causality[1][0]['params_ftest'][0]) + "; p="+ "{:.4f}".format(G_causality[1][0]['params_ftest'][1]))
        if G_causality[1][0]['params_ftest'][1] < 0.001:
            print(j+' is a cause of '+i)


F test for temperature (degC)
F=535.6704; p=0.0000
temperature (degC) is a cause of Solar3
F test for dewpoint_temperature (degC)
F=1.2089; p=0.2716
F test for wind_speed (m/s)
F=79.6777; p=0.0000
wind_speed (m/s) is a cause of Solar3
F test for mean_sea_level_pressure (Pa)
F=26.1359; p=0.0000
mean_sea_level_pressure (Pa) is a cause of Solar3
F test for relative_humidity ((0-1))
F=1057.8022; p=0.0000
relative_humidity ((0-1)) is a cause of Solar3
F test for surface_solar_radiation (W/m^2)
F=838.7920; p=0.0000
surface_solar_radiation (W/m^2) is a cause of Solar3
F test for surface_thermal_radiation (W/m^2)
F=76.6195; p=0.0000
surface_thermal_radiation (W/m^2) is a cause of Solar3
F test for total_cloud_cover (0-1)
F=10.4444; p=0.0012


In [15]:
"{:.4f}".format(G_causality[5][0]['params_ftest'][0])

'96.7930'

In [77]:
G_causality[5][0]['params_ftest'][1]

8.655891501276187e-101

In [8]:
data_cleanned_v3 = read_pickle(path_name = 'data\data_processed_experiment.pickle')

In [82]:
data_cleanned_v2['Solar1'].keys()[1:]

Index(['temperature (degC)', 'dewpoint_temperature (degC)', 'wind_speed (m/s)',
       'mean_sea_level_pressure (Pa)', 'relative_humidity ((0-1))',
       'surface_solar_radiation (W/m^2)', 'surface_thermal_radiation (W/m^2)',
       'total_cloud_cover (0-1)'],
      dtype='object')

In [17]:
data_cleanned_v3['Solar0']

,demand(kWh)/production(kW),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
2020-04-25 14:00:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:15:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:30:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:45:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 15:00:00,0.00,16.02,14.10,5.80,101051.09,0.88,0.00,368.64,1.00
...,...,...,...,...,...,...,...,...,...
2020-11-30 22:45:00,13.21,23.56,14.05,6.34,100293.63,0.55,268.04,365.36,0.93
2020-11-30 23:00:00,8.24,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:15:00,9.42,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:30:00,7.63,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99


In [18]:
grangercausalitytests(data_cleanned_v3['Solar0'][['temperature (degC)', 'demand(kWh)/production(kW)']], maxlag=[3])


Granger Causality
number of lags (no zero) 3
ssr based F test:         F=834.9748, p=0.0000  , df_denom=21054, df_num=3
ssr based chi2 test:   chi2=2505.7572, p=0.0000  , df=3
likelihood ratio test: chi2=2367.5541, p=0.0000  , df=3
parameter F test:         F=834.9748, p=0.0000  , df_denom=21054, df_num=3


{3: ({'ssr_ftest': (834.9747878545519, 0.0, 21054.0, 3),
   'ssr_chi2test': (2505.7571967803815, 0.0, 3),
   'lrtest': (2367.554133359532, 0.0, 3),
   'params_ftest': (834.9747878543955, 0.0, 21054.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0.]])])}

## Trail on time series GAN

In [9]:
data_cleanned_v3['Solar0']

,demand(kWh)/production(kW),temperature (degC),dewpoint_temperature (degC),wind_speed (m/s),mean_sea_level_pressure (Pa),relative_humidity ((0-1)),surface_solar_radiation (W/m^2),surface_thermal_radiation (W/m^2),total_cloud_cover (0-1)
2020-04-25 14:00:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:15:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:30:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 14:45:00,0.00,16.14,13.61,6.27,101156.43,0.85,0.00,366.92,1.00
2020-04-25 15:00:00,0.00,16.02,14.10,5.80,101051.09,0.88,0.00,368.64,1.00
...,...,...,...,...,...,...,...,...,...
2020-11-30 22:45:00,13.21,23.56,14.05,6.34,100293.63,0.55,268.04,365.36,0.93
2020-11-30 23:00:00,8.24,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:15:00,9.42,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99
2020-11-30 23:30:00,7.63,24.29,14.34,6.36,100331.94,0.54,236.50,374.79,0.99


In [9]:
from sklearn.preprocessing import MinMaxScaler
def real_data_loading(data: np.array, seq_len):
    """Load and preprocess real-world datasets.
    Args:
      - data_name: Numpy array with the values from a a Dataset
      - seq_len: sequence length

    Returns:
      - data: preprocessed data.
    """
    # Flip the data to make chronological data
    ori_data = data[::-1]
    # Normalize the data
    scaler = MinMaxScaler().fit(ori_data)
    ori_data = scaler.transform(ori_data)

    # Preprocess the dataset
    temp_data = []
    # Cut data by sequence length
    for i in range(0, len(ori_data) - seq_len):
        _x = ori_data[i:i + seq_len]
        temp_data.append(_x)

    # Mix the datasets (to make it similar to i.i.d)
    idx = np.random.permutation(len(temp_data))
    data = []
    for i in range(len(temp_data)):
        data.append(temp_data[idx[i]])
    return data

In [10]:
loaded_Data = real_data_loading(data_cleanned_v3['Solar0'], 96)

In [11]:
len(loaded_Data)

20968

In [17]:
from typeguard import typechecked
from collections import namedtuple
from typing import List, Optional, Union
import tqdm
from joblib import dump, load
from numpy import array, vstack
from pandas import DataFrame
from tensorflow import config as tfconfig

_model_parameters = ['batch_size', 'lr', 'betas', 'layers_dim', 'noise_dim',
                     'n_cols', 'seq_len', 'condition', 'n_critic', 'n_features']
_model_parameters_df = [128, 1e-4, (None, None), 128, 264,
                        None, None, None, 1, None]

_train_parameters = ['cache_prefix', 'label_dim', 'epochs', 'sample_interval', 'labels']
ModelParameters = namedtuple('ModelParameters', _model_parameters, defaults=_model_parameters_df)
TrainParameters = namedtuple('TrainParameters', _train_parameters, defaults=('', None, 300, 50, None))

@typechecked
class BaseModel():
    """
    Base class of GAN synthesizer models.
    The main methods are train (for fitting the synthesizer), save/load and sample (obtain synthetic records).
    Args:
        model_parameters (ModelParameters):
            Set of architectural parameters for model definition.
    """
    __MODEL__ = None

    def __init__(
            self,
            model_parameters: ModelParameters
    ):
        gpu_devices = tfconfig.list_physical_devices('GPU')
        if len(gpu_devices) > 0:
            try:
                tfconfig.experimental.set_memory_growth(gpu_devices[0], True)
            except (ValueError, RuntimeError):
                # Invalid device or cannot modify virtual devices once initialized.
                pass
        #Validate the provided model parameters
        if model_parameters.betas is not None:
            assert len(model_parameters.betas) == 2, "Please provide the betas information as a tuple."

        self.batch_size = model_parameters.batch_size
        self._set_lr(model_parameters.lr)
        self.beta_1 = model_parameters.betas[0]
        self.beta_2 = model_parameters.betas[1]
        self.noise_dim = model_parameters.noise_dim
        self.data_dim = None
        self.layers_dim = model_parameters.layers_dim
        self.processor = None

    # pylint: disable=E1101
    def __call__(self, inputs, **kwargs):
        return self.model(inputs=inputs, **kwargs)

    # pylint: disable=C0103
    def _set_lr(self, lr):
        if isinstance(lr, float):
            self.g_lr=lr
            self.d_lr=lr
        elif isinstance(lr,(list, tuple)):
            assert len(lr)==2, "Please provide a tow values array for the learning rates or a float."
            self.g_lr=lr[0]
            self.d_lr=lr[1]

    def define_gan(self):
        """Define the trainable model components.
        Optionally validate model structure with mock inputs and initialize optimizers."""
        raise NotImplementedError

    @property
    def model_parameters(self):
        "Returns the parameters of the model."
        return self._model_parameters

    @property
    def model_name(self):
        "Returns the model (class) name."
        return self.__class__.__name__

    def train(self,
              data: Union[DataFrame, array],
              num_cols: Optional[List[str]] = None,
              cat_cols: Optional[List[str]] = None) -> Union[DataFrame, array]:
        """Sets up the train session by instantiating an appropriate processor, fitting and storing it as an attribute.
        Args:
            data (Union[DataFrame, array]): Raw data object.
            num_cols (Optional[List[str]]): List of names of numerical columns.
            cat_cols (Optional[List[str]]): List of names of categorical columns.
        """
        if self.__MODEL__ in RegularModels.__members__:
            self.processor = RegularDataProcessor
        elif self.__MODEL__ in TimeSeriesModels.__members__:
            self.processor = TimeSeriesDataProcessor
        else:
            print(f'A DataProcessor is not available for the {self.__MODEL__}.')
        self.processor = self.processor(num_cols = num_cols, cat_cols = cat_cols).fit(data)

    def sample(self, n_samples: int):
        """Generate n_samples synthetic records from the synthesizer.
        The records returned are always a multiple of batch_size (can return excess of up to batch_size - 1 records).
        The samples are returned in the original data format, with any internal preprocessing inverted.

        Args:
            n_samples (int): Intended size of the synthetic sample.
        """
        steps = n_samples // self.batch_size + 1
        data = []
        for _ in tqdm.trange(steps, desc='Synthetic data generation'):
            z = tf.random.uniform([self.batch_size, self.noise_dim], dtype=tf.dtypes.float32)
            records = self.generator(z, training=False).numpy()
            data.append(records)
        return self.processor.inverse_transform(array(vstack(data)))

    def save(self, path):
        "Saves the pickled synthesizer instance in the given path."
        #Save only the generator?
        if self.__MODEL__=='WGAN' or self.__MODEL__=='WGAN_GP':
            del self.critic
        make_keras_picklable()
        dump(self, path)

    @staticmethod
    def load(path):
        "Loads a pickled synthesizer from the given path."
        gpu_devices = tf.config.list_physical_devices('GPU')
        if len(gpu_devices) > 0:
            try:
                tfconfig.experimental.set_memory_growth(gpu_devices[0], True)
            except (ValueError, RuntimeError):
                # Invalid device or cannot modify virtual devices once initialized.
                pass
        synth = load(path)
        return synth


In [45]:
gan_args

ModelParameters(batch_size=128, lr=0.0005, betas=(None, None), layers_dim=128, noise_dim=32, n_cols=None, seq_len=None, condition=None, n_critic=1, n_features=None)

In [28]:
from tensorflow import function, GradientTape, sqrt, abs, reduce_mean, ones_like, zeros_like, convert_to_tensor,float32
from tensorflow import data as tfdata
from tensorflow import nn
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.layers import GRU, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tqdm import tqdm
from typeguard import typechecked
from collections import namedtuple
from typing import List, Optional, Union

_model_parameters = ['batch_size', 'lr', 'betas', 'layers_dim', 'noise_dim',
                     'n_cols', 'seq_len', 'condition', 'n_critic', 'n_features']
_model_parameters_df = [128, 1e-4, (None, None), 128, 264,
                        None, None, None, 1, None]

_train_parameters = ['cache_prefix', 'label_dim', 'epochs', 'sample_interval', 'labels']
ModelParameters = namedtuple('ModelParameters', _model_parameters, defaults=_model_parameters_df)
TrainParameters = namedtuple('TrainParameters', _train_parameters, defaults=('', None, 300, 50, None))

class TimeGAN():

    __MODEL__='TimeGAN'

    def __init__(self, model_parameters, hidden_dim, seq_len, n_seq, gamma):
        self.seq_len=seq_len
        self.n_seq=n_seq
        self.hidden_dim=hidden_dim
        self.gamma=gamma
        self.batch_size = model_parameters.batch_size
        self.lr = model_parameters.lr
        self.g_lr = model_parameters.lr
        self.d_lr = model_parameters.lr
        self.layers_dim = model_parameters.layers_dim
        self.noise_dim = model_parameters.noise_dim
        # super().__init__(model_parameters)

    def define_gan(self):
        self.generator_aux=Generator(self.hidden_dim).build()
        self.supervisor=Supervisor(self.hidden_dim).build()
        self.discriminator=Discriminator(self.hidden_dim).build()
        self.recovery = Recovery(self.hidden_dim, self.n_seq).build()
        self.embedder = Embedder(self.hidden_dim).build()

        X = Input(shape=[self.seq_len, self.n_seq], batch_size=self.batch_size, name='RealData')
        Z = Input(shape=[self.seq_len, self.n_seq], batch_size=self.batch_size, name='RandomNoise')

        #--------------------------------
        # Building the AutoEncoder
        #--------------------------------
        H = self.embedder(X)
        X_tilde = self.recovery(H)

        self.autoencoder = Model(inputs=X, outputs=X_tilde)

        #---------------------------------
        # Adversarial Supervise Architecture
        #---------------------------------
        E_Hat = self.generator_aux(Z)
        H_hat = self.supervisor(E_Hat)
        Y_fake = self.discriminator(H_hat)

        self.adversarial_supervised = Model(inputs=Z,
                                       outputs=Y_fake,
                                       name='AdversarialSupervised')

        #---------------------------------
        # Adversarial architecture in latent space
        #---------------------------------
        Y_fake_e = self.discriminator(E_Hat)

        self.adversarial_embedded = Model(inputs=Z,
                                    outputs=Y_fake_e,
                                    name='AdversarialEmbedded')
        # ---------------------------------
        # Synthetic data generation
        # ---------------------------------
        X_hat = self.recovery(H_hat)
        self.generator = Model(inputs=Z,
                            outputs=X_hat,
                            name='FinalGenerator')

        # --------------------------------
        # Final discriminator model
        # --------------------------------
        Y_real = self.discriminator(H)
        self.discriminator_model = Model(inputs=X,
                                         outputs=Y_real,
                                         name="RealDiscriminator")

        # ----------------------------
        # Define the loss functions
        # ----------------------------
        self._mse=MeanSquaredError()
        self._bce=BinaryCrossentropy()


    @function
    def train_autoencoder(self, x, opt):
        with GradientTape() as tape:
            x_tilde = self.autoencoder(x)
            embedding_loss_t0 = self._mse(x, x_tilde)
            e_loss_0 = 10 * sqrt(embedding_loss_t0)

        var_list = self.embedder.trainable_variables + self.recovery.trainable_variables
        gradients = tape.gradient(e_loss_0, var_list)
        opt.apply_gradients(zip(gradients, var_list))
        return sqrt(embedding_loss_t0)

    @function
    def train_supervisor(self, x, opt):
        with GradientTape() as tape:
            h = self.embedder(x)
            h_hat_supervised = self.supervisor(h)
            generator_loss_supervised = self._mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

        var_list = self.supervisor.trainable_variables + self.generator.trainable_variables
        gradients = tape.gradient(generator_loss_supervised, var_list)
        apply_grads = [(grad, var) for (grad, var) in zip(gradients, var_list) if grad is not None]
        opt.apply_gradients(apply_grads)
        return generator_loss_supervised

    @function
    def train_embedder(self,x, opt):
        with GradientTape() as tape:
            # Supervised Loss
            h = self.embedder(x)
            h_hat_supervised = self.supervisor(h)
            generator_loss_supervised = self._mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

            # Reconstruction Loss
            x_tilde = self.autoencoder(x)
            embedding_loss_t0 = self._mse(x, x_tilde)
            e_loss = 10 * sqrt(embedding_loss_t0) + 0.1 * generator_loss_supervised

        var_list = self.embedder.trainable_variables + self.recovery.trainable_variables
        gradients = tape.gradient(e_loss, var_list)
        opt.apply_gradients(zip(gradients, var_list))
        return sqrt(embedding_loss_t0)

    def discriminator_loss(self, x, z):
        # Loss on false negatives
        y_real = self.discriminator_model(x)
        discriminator_loss_real = self._bce(y_true=ones_like(y_real),
                                            y_pred=y_real)

        # Loss on false positives
        y_fake = self.adversarial_supervised(z)
        discriminator_loss_fake = self._bce(y_true=zeros_like(y_fake),
                                            y_pred=y_fake)

        y_fake_e = self.adversarial_embedded(z)
        discriminator_loss_fake_e = self._bce(y_true=zeros_like(y_fake_e),
                                              y_pred=y_fake_e)
        return (discriminator_loss_real +
                discriminator_loss_fake +
                self.gamma * discriminator_loss_fake_e)

    @staticmethod
    def calc_generator_moments_loss(y_true, y_pred):
        y_true_mean, y_true_var = nn.moments(x=y_true, axes=[0])
        y_pred_mean, y_pred_var = nn.moments(x=y_pred, axes=[0])
        g_loss_mean = reduce_mean(abs(y_true_mean - y_pred_mean))
        g_loss_var = reduce_mean(abs(sqrt(y_true_var + 1e-6) - sqrt(y_pred_var + 1e-6)))
        return g_loss_mean + g_loss_var

    @function
    def train_generator(self, x, z, opt):
        with GradientTape() as tape:
            y_fake = self.adversarial_supervised(z)
            generator_loss_unsupervised = self._bce(y_true=ones_like(y_fake),
                                                    y_pred=y_fake)

            y_fake_e = self.adversarial_embedded(z)
            generator_loss_unsupervised_e = self._bce(y_true=ones_like(y_fake_e),
                                                      y_pred=y_fake_e)
            h = self.embedder(x)
            h_hat_supervised = self.supervisor(h)
            generator_loss_supervised = self._mse(h[:, 1:, :], h_hat_supervised[:, :-1, :])

            x_hat = self.generator(z)
            generator_moment_loss = self.calc_generator_moments_loss(x, x_hat)

            generator_loss = (generator_loss_unsupervised +
                              generator_loss_unsupervised_e +
                              100 * sqrt(generator_loss_supervised) +
                              100 * generator_moment_loss)

        var_list = self.generator_aux.trainable_variables + self.supervisor.trainable_variables
        gradients = tape.gradient(generator_loss, var_list)
        opt.apply_gradients(zip(gradients, var_list))
        return generator_loss_unsupervised, generator_loss_supervised, generator_moment_loss

    @function
    def train_discriminator(self, x, z, opt):
        with GradientTape() as tape:
            discriminator_loss = self.discriminator_loss(x, z)

        var_list = self.discriminator.trainable_variables
        gradients = tape.gradient(discriminator_loss, var_list)
        opt.apply_gradients(zip(gradients, var_list))
        return discriminator_loss

    def get_batch_data(self, data, n_windows):
        data = convert_to_tensor(data, dtype=float32)
        return iter(tfdata.Dataset.from_tensor_slices(data)
                                .shuffle(buffer_size=n_windows)
                                .batch(self.batch_size).repeat())

    def _generate_noise(self):
        while True:
            yield np.random.uniform(low=0, high=1, size=(self.seq_len, self.n_seq))

    def get_batch_noise(self):
        return iter(tfdata.Dataset.from_generator(self._generate_noise, output_types=float32)
                                .batch(self.batch_size)
                                .repeat())

    def train(self, data, train_steps):
        # Assemble the model
        self.define_gan()

        ## Embedding network training
        autoencoder_opt = Adam(learning_rate=self.g_lr)
        for _ in tqdm(range(train_steps), desc='Emddeding network training'):
            X_ = next(self.get_batch_data(data, n_windows=len(data)))
            step_e_loss_t0 = self.train_autoencoder(X_, autoencoder_opt)

        ## Supervised Network training
        supervisor_opt = Adam(learning_rate=self.g_lr)
        for _ in tqdm(range(train_steps), desc='Supervised network training'):
            X_ = next(self.get_batch_data(data, n_windows=len(data)))
            step_g_loss_s = self.train_supervisor(X_, supervisor_opt)

        ## Joint training
        generator_opt = Adam(learning_rate=self.g_lr)
        embedder_opt = Adam(learning_rate=self.g_lr)
        discriminator_opt = Adam(learning_rate=self.d_lr)

        step_g_loss_u = step_g_loss_s = step_g_loss_v = step_e_loss_t0 = step_d_loss = 0
        for _ in tqdm(range(train_steps), desc='Joint networks training'):

            #Train the generator (k times as often as the discriminator)
            # Here k=2
            for _ in range(2):
                X_ = next(self.get_batch_data(data, n_windows=len(data)))
                Z_ = next(self.get_batch_noise())
                # --------------------------
                # Train the generator
                # --------------------------
                step_g_loss_u, step_g_loss_s, step_g_loss_v = self.train_generator(X_, Z_, generator_opt)

                # --------------------------
                # Train the embedder
                # --------------------------
                step_e_loss_t0 = self.train_embedder(X_, embedder_opt)

            X_ = next(self.get_batch_data(data, n_windows=len(data)))
            Z_ = next(self.get_batch_noise())
            step_d_loss = self.discriminator_loss(X_, Z_)
            if step_d_loss > 0.15:
                step_d_loss = self.train_discriminator(X_, Z_, discriminator_opt)

    def sample(self, n_samples):
        steps = n_samples // self.batch_size + 1
        data = []
        for _ in tqdm.trange(steps, desc='Synthetic data generation'):
            Z_ = next(self.get_batch_noise())
            records = self.generator(Z_)
            data.append(records)
        return np.array(np.vstack(data))

class Generator(Model):
    def __init__(self, hidden_dim, net_type='GRU'):
        self.hidden_dim = hidden_dim
        self.net_type = net_type

    def build(self):
        model = Sequential(name='Generator')
        model = make_net(model,
                         n_layers=3,
                         hidden_units=self.hidden_dim,
                         output_units=self.hidden_dim,
                         net_type=self.net_type)
        return model

class Discriminator(Model):
    def __init__(self, hidden_dim, net_type='GRU'):
        self.hidden_dim = hidden_dim
        self.net_type=net_type

    def build(self):
        model = Sequential(name='Discriminator')
        model = make_net(model,
                         n_layers=3,
                         hidden_units=self.hidden_dim,
                         output_units=1,
                         net_type=self.net_type)
        return model

class Recovery(Model):
    def __init__(self, hidden_dim, n_seq):
        self.hidden_dim=hidden_dim
        self.n_seq=n_seq
        return

    def build(self):
        recovery = Sequential(name='Recovery')
        recovery = make_net(recovery,
                            n_layers=3,
                            hidden_units=self.hidden_dim,
                            output_units=self.n_seq)
        return recovery

class Embedder(Model):

    def __init__(self, hidden_dim):
        self.hidden_dim=hidden_dim
        return

    def build(self):
        embedder = Sequential(name='Embedder')
        embedder = make_net(embedder,
                            n_layers=3,
                            hidden_units=self.hidden_dim,
                            output_units=self.hidden_dim)
        return embedder

class Supervisor(Model):
    def __init__(self, hidden_dim):
        self.hidden_dim=hidden_dim

    def build(self):
        model = Sequential(name='Supervisor')
        model = make_net(model,
                         n_layers=2,
                         hidden_units=self.hidden_dim,
                         output_units=self.hidden_dim)
        return model
def make_net(model, n_layers, hidden_units, output_units, net_type='GRU'):
    if net_type=='GRU':
        for i in range(n_layers):
            model.add(GRU(units=hidden_units,
                      return_sequences=True,
                      name=f'GRU_{i + 1}'))
    else:
        for i in range(n_layers):
            model.add(LSTM(units=hidden_units,
                      return_sequences=True,
                      name=f'LSTM_{i + 1}'))

    model.add(Dense(units=output_units,
                    activation='sigmoid',
                    name='OUT'))
    return model


In [24]:
seq_len=96
n_seq = 6
hidden_dim=24
gamma=1

noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
learning_rate = 5e-4

gan_args = ModelParameters(batch_size=batch_size,
                           lr=learning_rate,
                           noise_dim=noise_dim,
                           layers_dim=dim)


synth = TimeGAN(model_parameters=gan_args, hidden_dim=24, seq_len=seq_len, n_seq=9, gamma=1)
synth.train(loaded_Data, train_steps=1000)  

Joint networks training: 100%|██████████| 1000/1000 [3:20:38<00:00, 12.04s/it] 


In [31]:
synth_data = synth.sample(960)
synth_data

Synthetic data generation: 100%|██████████| 8/8 [00:05<00:00,  1.60it/s]


array([[[8.17539096e-02, 3.53591859e-01, 4.17502761e-01, ...,
         6.82903230e-02, 4.20390487e-01, 6.03754699e-01],
        [5.64460158e-02, 3.81440669e-01, 4.45712835e-01, ...,
         9.83281136e-02, 3.99632007e-01, 5.35161555e-01],
        [5.69300056e-02, 4.20416296e-01, 4.87002164e-01, ...,
         1.47982657e-01, 4.60356206e-01, 5.96604347e-01],
        ...,
        [4.00194526e-03, 2.63870537e-01, 3.95279676e-01, ...,
         2.35366821e-03, 4.10865963e-01, 9.44450021e-01],
        [2.72625685e-03, 2.53280789e-01, 3.48681092e-01, ...,
         1.79472566e-03, 4.11369503e-01, 9.37144279e-01],
        [1.90153718e-03, 2.55237401e-01, 3.48038554e-01, ...,
         1.52802467e-03, 4.77768958e-01, 9.34035301e-01]],

       [[5.27192056e-02, 2.83431023e-01, 3.76851857e-01, ...,
         3.45009863e-02, 3.81548107e-01, 6.70689702e-01],
        [2.20795572e-02, 2.92908072e-01, 4.00556207e-01, ...,
         3.11471522e-02, 3.53015661e-01, 6.41106248e-01],
        [1.94412470e-02, 

In [30]:
import tqdm
from tqdm import  trange

In [32]:
synth_data[0]

array([[8.17539096e-02, 3.53591859e-01, 4.17502761e-01, 3.49955678e-01,
        7.66160190e-01, 7.30810523e-01, 6.82903230e-02, 4.20390487e-01,
        6.03754699e-01],
       [5.64460158e-02, 3.81440669e-01, 4.45712835e-01, 3.79688740e-01,
        7.65500546e-01, 6.91188455e-01, 9.83281136e-02, 3.99632007e-01,
        5.35161555e-01],
       [5.69300056e-02, 4.20416296e-01, 4.87002164e-01, 4.79326606e-01,
        7.19001770e-01, 6.56040668e-01, 1.47982657e-01, 4.60356206e-01,
        5.96604347e-01],
       [6.05844259e-02, 4.78663117e-01, 5.39811015e-01, 5.31581163e-01,
        6.52343512e-01, 5.90288520e-01, 2.23566234e-01, 5.37684679e-01,
        5.83547115e-01],
       [8.36847425e-02, 5.33392310e-01, 5.77805400e-01, 5.50294459e-01,
        5.78352690e-01, 5.48484862e-01, 3.19206417e-01, 5.69276333e-01,
        4.70828176e-01],
       [1.50101721e-01, 5.59132099e-01, 5.95506310e-01, 5.15473008e-01,
        5.39443552e-01, 5.42187512e-01, 4.32796180e-01, 5.48629045e-01,
        2.9

In [76]:
fig = go.Figure()
test_length = 96
fig.add_trace(
    go.Scatter(
        x= data_cleanned_v3['Solar0']['demand(kWh)/production(kW)'].index[:test_length], y=synth_geneartion[0], line_color= 'blue', line_width = 0.8, opacity = 0.7, name = 'Solar geneartion'
    )
)

fig.add_trace(
    go.Scatter(
        x= data_cleanned_v3['Solar0']['demand(kWh)/production(kW)'].index[:test_length], y=data_cleanned_v3['Solar0']['demand(kWh)/production(kW)'][test_length:2*test_length], line_color= 'red', line_width = 0.8, opacity = 0.7, name = 'Solar geneartion training set'
    )
)
fig.layout.xaxis.title = "Time"
fig.layout.yaxis.title = "Hourly Imported ELectricity (kW)"
fig.layout.plot_bgcolor='rgba(0,0,0,0)'
fig.layout.font.family="Times New Roman" ## "Times New Roman Black"
fig.layout.font.size = 20
fig.layout.width = 1000
fig.layout.height = 500
fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True,showgrid=False, tickangle = 45 )
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, showgrid=False)
fig.show()

In [80]:
data_cleanned_v3['Solar0']['demand(kWh)/production(kW)'][1057:1200]

2020-05-06 14:15:00    0.0
2020-05-06 14:30:00    0.0
2020-05-06 14:45:00    0.0
2020-05-06 15:00:00    0.0
2020-05-06 15:15:00    0.0
                      ... 
2020-05-08 00:45:00    0.0
2020-05-08 01:00:00    0.0
2020-05-08 01:15:00    0.0
2020-05-08 01:30:00    0.0
2020-05-08 01:45:00    0.0
Name: demand(kWh)/production(kW), Length: 143, dtype: float64

In [65]:
synth_data[0][0][0]

0.08175391

In [51]:
data_cleanned_v3['Solar0']['demand(kWh)/production(kW)'][:test_length]

2020-04-25 14:00:00    0.0
2020-04-25 14:15:00    0.0
2020-04-25 14:30:00    0.0
2020-04-25 14:45:00    0.0
2020-04-25 15:00:00    0.0
                      ... 
2020-04-26 12:45:00    0.0
2020-04-26 13:00:00    0.0
2020-04-26 13:15:00    0.0
2020-04-26 13:30:00    0.0
2020-04-26 13:45:00    0.0
Name: demand(kWh)/production(kW), Length: 96, dtype: float64

In [66]:
synth_data.shape

(1024, 96, 9)

In [73]:
synth_geneartion = synth_data[:,:,0]

In [74]:
synth_geneartion[0]

array([8.17539096e-02, 5.64460158e-02, 5.69300056e-02, 6.05844259e-02,
       8.36847425e-02, 1.50101721e-01, 2.77018845e-01, 3.27947646e-01,
       2.18120635e-01, 1.11342400e-01, 6.31257892e-02, 4.76890206e-02,
       4.16274667e-02, 2.96477675e-02, 1.82679892e-02, 1.20614767e-02,
       1.11820400e-02, 1.18908882e-02, 1.57601833e-02, 2.25926042e-02,
       2.59434879e-02, 2.79764533e-02, 3.08025777e-02, 2.68412232e-02,
       1.60510242e-02, 7.90178776e-03, 3.26102972e-03, 1.60163641e-03,
       9.17375088e-04, 8.23616982e-04, 1.25825405e-03, 2.74842978e-03,
       6.15790486e-03, 1.22595131e-02, 1.87210143e-02, 2.31209099e-02,
       2.87624002e-02, 3.35291624e-02, 4.35833037e-02, 4.26331162e-02,
       2.65238881e-02, 1.29700899e-02, 5.32814860e-03, 2.04747915e-03,
       1.00165606e-03, 1.04862452e-03, 1.95193291e-03, 4.10491228e-03,
       6.87819719e-03, 8.64139199e-03, 8.72474909e-03, 8.09058547e-03,
       7.93105364e-03, 9.59241390e-03, 1.79575086e-02, 4.63046432e-02,
      